In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud


In [4]:
spotify_stream_df = pd.read_json("MyData/StreamingHistory0.json")
spotify_stream_df.tail()

,endTime,artistName,trackName,msPlayed
8836,2023-05-05 14:21,A$AP Ferg,"Dreams, Fairytales, Fantasies (feat. Brent Fai...",709
8837,2023-05-05 14:21,A$AP Ferg,Floor Seats,0
8838,2023-05-05 14:21,Kendrick Lamar,u,833
8839,2023-05-05 14:26,Baby Keem,trademark usa,270670
8840,2023-05-05 14:27,A$AP Ferg,Floor Seats,70224


In [5]:
spotify_stream_df.shape

(8841, 4)

In [6]:
spotify_stream_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8841 entries, 0 to 8840
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   endTime     8841 non-null   object
 1   artistName  8841 non-null   object
 2   trackName   8841 non-null   object
 3   msPlayed    8841 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 276.4+ KB


In [7]:
spotify_stream_df.nunique() # To see the unique elemetns present in each column

endTime       4667
artistName     695
trackName     1816
msPlayed      4715
dtype: int64

In [8]:
spotify_stream_df.sample(10)

,endTime,artistName,trackName,msPlayed
5538,2023-03-16 02:47,2Pac,All Eyez On Me (ft. Big Syke),954
1523,2023-01-01 05:29,Kendrick Lamar,Now Or Never - Bonus Track,4602
3858,2023-02-11 20:39,Genjutsu Beats,Sparta,0
4781,2023-02-24 15:14,King,Ektarfa,14089
1203,2022-12-26 10:58,Unknown Artist,Unknown Track,721
7119,2023-04-05 18:08,Rashid Ali,Kabhi Kabhi Aditi,218836
6496,2023-03-27 08:30,Sidharth Basrur,Mujhe De De Har Gham Tera,25727
5604,2023-03-17 14:31,Joji,Die For You,211666
1886,2023-01-09 11:27,Bas,[The Jackie] (with J. Cole & Lil Tjay),3303
6695,2023-03-30 04:44,Lil Durk,JUMP (feat. Memo600),192718


In [9]:
len(spotify_stream_df["artistName"].unique()) # Length of unique artist

695

there's 695 number of unique artist in my streaming history